## 链表-LinkedList

### 单链表(SinglyLinedList)实现

链表表有以下特性：
- O(1)的插入时间和删除时间
- O(N)的查找时间
和Array基本上是互补的， 和Array最大的不同在于， 它不需要在内存空间中有一块连续的地址， 而Array往往是需要一段连续地址空间；意味着链表可以更大化地利用内存空间， 当然代价是造成内存空间的碎片化，会频繁触发垃圾回收影响性能。
除了单链表以外， 它的变形还包括：
- 循坏链表， 也就是链表的尾部指向头部
- 双向链表， 链表中的node除了保留对下一个节点的引用以外， 还保留了对前一个节点的应用， 进而在有序链表的情况下， 缩短查找时间

In [9]:
from typing import Optional


class Node:
    def __init__(self, data: int, next_node=None):
        self.data = data
        self._next = next_node

class SinglyLinkedList:
    def __init__(self):
        self._head = None

    def find_by_value(self, value: int) -> Optional[Node]:
        p = self._head
        while p and p.data != value:
            p = p._next
        if p:
            return p
        raise ValueError(f"Value {value} not found in SinglyLinkedList")

    def find_by_index(self, index: int) -> Optional[Node]:
        p = self._head
        position = 0
        while p and position != index:
            p = p._next
            position += 1
        return p

    def insert_value_to_head(self, value: int):
        new_node = Node(value)
        self.insert_node_to_head(new_node)

    def insert_node_to_head(self, new_node: Node):
        new_node._next = self._head
        self._head = new_node

    def insert_value_after(self, node: Node, value: int):
        new_node = Node(value)
        self.insert_node_after(node, new_node)

    def insert_node_after(self, node: Node, new_node: Node):
        new_node._next = node._next
        node._next = new_node

    def insert_value_before(self, node: Node, value: int):
        new_node = Node(value)
        self.insert_node_before(node, new_node)

    def insert_node_before(self, node: Node, new_node: Node):
        if not self._head:
            return
        if self._head == node:
            self.insert_node_to_head(new_node)
            return
        current = self._head
        while current._next and current._next != node:
            current = current._next
        if not current._next:  # node is not even in the list
            return
        new_node._next = node
        current._next = new_node

    def delete_by_node(self, node: Node):
        # 当链表里面没有数据的时候
        if not self._head:
            return
        # 当node有下一个node的时候， 直接更新node的data和它的下一个引用
        if node._next:
            node.data = node._next.data
            node._next = node._next._next
            return
        # 当node没有下一个node的时候
        current = self._head
        while current and current._next != node:
            current = current._next
        if not current: 
            return
        current._next = node._next

    def delete_by_value(self, value: int):
        if not self._head or not value:
            return
        fake_head = Node(value + 1)
        fake_head._next = self._head
        prev, current = fake_head, self._head
        while current:
            if current.data != value:
                prev._next = current
                prev = prev._next
            current = current._next
        if prev._next:
            prev._next = None
        self._head = fake_head._next  

    def print_all(self):
        current = self._head
        if current:
            print(f"{current.data}", end="")
            current = current._next
        while current:
            print(f"->{current.data}", end="")
            current = current._next
        print("\n", flush=True)

    def __repr__(self) -> str:
        nums = []
        current = self._head
        while current:
            nums.append(current.data)
            current = current._next
        return "->".join(str(num) for num in nums)

    def __iter__(self):
        node = self._head
        while node:
            yield node.data
            node = node._next


#### Demo

In [16]:
l = SinglyLinkedList()
for i in range(15):
    l.insert_value_to_head(i)
node9 = l.find_by_value(9)
l.insert_value_before(node9, 20)
l.insert_value_before(node9, 16)
l.insert_value_before(node9, 16)
l.delete_by_value(16)
node11 = l.find_by_index(3)
l.delete_by_node(node11)
l.delete_by_node(l._head)
l.delete_by_value(13)
print(l)
for value in l:
    print(value)


12->10->20->9->8->7->6->5->4->3->2->1->0
12
10
20
9
8
7
6
5
4
3
2
1
0


#### 常见算法应用

#### 实现LRU

In [1]:
class DbListNode(object):
    def __init__(self, x, y):
        self.key = x
        self.val = y
        self.next = None
        self.prev = None


class LRUCache:
    def __init__(self, capacity: int):
        self.cap = capacity
        self.hkeys = {}
        # self.top和self.tail作为哨兵节点, 避免越界
        self.top = DbListNode(None, -1)   # top节点的下一个节点是真正的headnode(第一个节点, 头结点)
        self.tail = DbListNode(None, -1)  # tail一开始充当的是headnode
        self.top.next = self.tail
        self.tail.prev = self.top

    def get(self, key: int) -> int:
        cur = self.hkeys.get(key,None)
        if cur:
            # 跳出原位置
            delete_cur_node(cur)
            # 最近用过的置于链表首部
            replace_headnode(self, cur)
            return self.hkeys[key].val
        return -1

    def put(self, key: int, value: int) -> None:
        cur = self.hkeys.get(key,None)
        if cur:
            cur.val = value
            # 跳出原位置
            delete_cur_node(cur)
            replace_headnode(self, cur)
        else:
            # 增加新结点至首部
            cur = DbListNode(key, value)
            self.hkeys[key] = cur   # {1 : node}
            replace_headnode(self, cur)
            if len(self.hkeys.keys()) > self.cap:
                self.hkeys.pop(self.tail.prev.key)
                # 去掉原尾结点
                self.tail.prev.prev.next = self.tail
                self.tail.prev = self.tail.prev.prev

    def __repr__(self):
        vals = []
        p = self.top.next
        while p.next:
            vals.append(str(p.val))
            p = p.next
        return '->'.join(vals)


def delete_cur_node(cur):
    '''
    删除当前节点
    '''
    cur.next.prev = cur.prev
    cur.prev.next = cur.next


def replace_headnode(self, cur):
    """
    将第一个head node替换为当前节点

    cur: 当前节点
    """
    top_node = self.top.next   # 一开始的是tail
    self.top.next = cur        # 变成cur
    cur.prev = self.top        #  
    cur.next = top_node        # top cur tail
    top_node.prev = cur        # 最开始的headnode的前一个变成了cur

if __name__ == '__main__':
    cache = LRUCache(2)
    cache.put(1, 1)
    cache.put(2, 2)
    print(cache)
    cache.get(1)  # 返回  1
    cache.put(3, 3)  # 该操作会使得密钥 2 作废
    print(cache)
    cache.get(2)  # 返回 -1 (未找到)
    cache.put(4, 4)  # 该操作会使得密钥 1 作废
    print(cache)
    cache.get(1)  # 返回 -1 (未找到)
    cache.get(3)  # 返回  3
    print(cache)
    cache.get(4)  # 返回  4
    print(cache)

2->1
3->1
4->3
3->4
4->3
